<a href="https://colab.research.google.com/github/biancviana/Bhaskara-em-C/blob/master/TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import numpy as np
import seaborn as sns
#!pip install skimpy

pd.set_option('display.max_columns', 40)
#pd.set_option('display.max_rows', 50)

In [7]:
from skimpy import skim

In [9]:
caminho_base_de_dados = r"/content/sample_data/bullying.csv"
df = pd.read_csv(caminho_base_de_dados, sep=';')

In [10]:
df.head()

,Escola,Curso,Cor,Genero,Ano,Idade,Demora dormir,Sonhos ruins,Qualidade do sono,Relação com Amigos,Relação com Professores,Auto-Imag,Fisico,Social,Verbal,Material,Virtual,Mora com pais,Escolari-mãe,Escolari-pai,Nota-exatas,Nota-humanas,Nota-biológicas,Tipo escola
0,ETIM,Integrado,Branca,Masculino,1,15,> 3 vezes por semana,0,Boa,Muito boa,Muito boa,Muito boa,0,0,0,1,0,Não,Ensino médio,Pós-graduação,4,2,3,Pública
1,ETIM,Integrado,Parda,Feminino,1,15,2-3 vezes por semana,2,Ruim,Boa,Regular,Boa,0,1,0,0,0,Sim,Ensino médio,Fundamental 1,3,4,3,Pública
2,ETIM,Integrado,Branca,Feminino,1,16,> 3 vezes por semana,1,Ruim,Boa,Boa,Ruim,0,1,2,0,2,Sim,Graduação,Graduação,3,5,4,Pública
3,ETIM,Integrado,Branca,Masculino,1,15,> 3 vezes por semana,0,Muito ruim,Muito boa,Boa,Boa,0,0,0,0,0,Sim,Ensino médio,Graduação,4,4,4,Pública
4,ETIM,Integrado,Branca,Feminino,1,15,1-2 vezes por semana,0,Boa,Boa,Boa,Boa,1,1,2,0,1,Sim,Graduação,Graduação,4,4,3,Pública


# ✅ **Preparação da base de dados**
Limpeza e pré-processamento


In [11]:
df_copy = df.copy()

## **1) Criação de nova coluna com o bullying total**

In [12]:
df_copy['Bullying Total'] = df_copy['Fisico'] + df_copy['Social'] + df_copy['Verbal'] + df_copy['Material'] + df_copy['Virtual']
df_copy.head()

,Escola,Curso,Cor,Genero,Ano,Idade,Demora dormir,Sonhos ruins,Qualidade do sono,Relação com Amigos,Relação com Professores,Auto-Imag,Fisico,Social,Verbal,Material,Virtual,Mora com pais,Escolari-mãe,Escolari-pai,Nota-exatas,Nota-humanas,Nota-biológicas,Tipo escola,Bullying Total
0,ETIM,Integrado,Branca,Masculino,1,15,> 3 vezes por semana,0,Boa,Muito boa,Muito boa,Muito boa,0,0,0,1,0,Não,Ensino médio,Pós-graduação,4,2,3,Pública,1
1,ETIM,Integrado,Parda,Feminino,1,15,2-3 vezes por semana,2,Ruim,Boa,Regular,Boa,0,1,0,0,0,Sim,Ensino médio,Fundamental 1,3,4,3,Pública,1
2,ETIM,Integrado,Branca,Feminino,1,16,> 3 vezes por semana,1,Ruim,Boa,Boa,Ruim,0,1,2,0,2,Sim,Graduação,Graduação,3,5,4,Pública,5
3,ETIM,Integrado,Branca,Masculino,1,15,> 3 vezes por semana,0,Muito ruim,Muito boa,Boa,Boa,0,0,0,0,0,Sim,Ensino médio,Graduação,4,4,4,Pública,0
4,ETIM,Integrado,Branca,Feminino,1,15,1-2 vezes por semana,0,Boa,Boa,Boa,Boa,1,1,2,0,1,Sim,Graduação,Graduação,4,4,3,Pública,5


In [13]:
mask = df_copy['Bullying Total'] > 0
df_pelo_menos_um_bullying = df_copy[mask]

df_copy.shape
df_pelo_menos_um_bullying.shape


print(f'Quantidade total de alunos: {df_copy.shape[0]}')
print(f'Quantidade de alunos que sofreram pelo menos um bullying: {df_pelo_menos_um_bullying.shape[0]}')
print(f'Quantidade de alunos que não sofreram bullying: {df_copy.shape[0] - df_pelo_menos_um_bullying.shape[0]}')

Quantidade total de alunos: 489
Quantidade de alunos que sofreram pelo menos um bullying: 363
Quantidade de alunos que não sofreram bullying: 126


## **2) Criação de nova coluna com bullying e sem bullying**

In [14]:
df_copy['Sofreu Bullying?'] = df_copy['Bullying Total']
df_copy['Sofreu Bullying?'].unique()

array([ 1,  5,  0,  8,  2,  7, 16,  4,  3,  6, 13, 12, 10,  9, 17, 11, 14,
       19, 18, 15, 20, 21, 26, 25, 22])

In [15]:
mask_bullying = df_copy['Sofreu Bullying?'] >= 1
df_copy.loc[mask_bullying, 'Sofreu Bullying?'] = 'Sofreu Bullying'

mask_sem_bullying = df_copy['Sofreu Bullying?'] == 0
df_copy.loc[mask_sem_bullying, 'Sofreu Bullying?'] = 'Não Sofreu Bullying'

df_copy['Sofreu Bullying?'].unique()

array(['Sofreu Bullying', 'Não Sofreu Bullying'], dtype=object)

In [16]:
#df_copy[['Bullying Total', 'Sofreu Bullying?']].sample(10)

In [17]:
fig = px.histogram(df_copy, x="Sofreu Bullying?")
fig.show()

## **3) Criação de nova coluna com a cor em dois grupos**

In [20]:
df_copy['Cor Agrupada'] = df_copy['Cor ']

mask_nao_branca = df_copy['Cor '] != 'Branca'
df_copy.loc[mask_nao_branca, 'Cor Agrupada'] = 'Não-Branca'

df_copy['Cor Agrupada'].unique()

array(['Branca', 'Não-Branca'], dtype=object)

In [ ]:
df_copy

In [ ]:
#df_copy[['Cor ', 'Cor Agrupada']]

# **Análise Descritiva**

In [44]:
df_copy.describe()

,Ano,Idade,Sonhos ruins,Fisico,Social,Verbal,Material,Virtual,Nota-exatas,Nota-humanas,Nota-biológicas,Bullying Total
count,489.00000,489.000000,489.000000,489.000000,489.000000,489.000000,489.000000,489.000000,489.000000,489.000000,489.000000,489.000000
mean,2.00409,16.848671,0.883436,0.730061,1.247444,1.822086,0.950920,0.584867,3.533742,3.916155,3.687117,5.335378
std,0.85170,1.263291,0.864078,1.280663,1.678614,2.000779,1.495089,1.225354,1.013933,0.770302,0.823737,5.922138
min,1.00000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,1.00000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,3.000000,3.000000,0.000000
50%,2.00000,17.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,4.000000,4.000000,4.000000,3.000000
75%,3.00000,18.000000,1.000000,1.000000,2.000000,3.000000,2.000000,0.000000,4.000000,4.000000,4.000000,9.000000
max,4.00000,19.000000,3.000000,5.000000,6.000000,6.000000,6.000000,6.000000,5.000000,5.000000,5.000000,26.000000


In [46]:
df_copy.describe(exclude=np.number)

,Escola,Curso,Cor,Genero,Demora dormir,Qualidade do sono,Relação com Amigos,Relação com Professores,Auto-Imag,Mora com pais,Escolari-mãe,Escolari-pai,Tipo escola,Sofreu Bullying?,Cor Agrupada
count,489,489,489,489,489,489,489,489,489,489,489,489,489,489,489
unique,7,2,3,2,4,5,5,5,5,2,6,5,2,2,2
top,ETIM,Integrado,Branca,Masculino,> 3 vezes por semana,Regular,Muito boa,Boa,Boa,Sim,Ensino médio,Ensino médio,Pública,Sofreu Bullying,Branca
freq,189,329,311,261,163,209,228,276,174,322,213,199,395,363,311


Ano                1.000000
Idade              0.461841
Sonhos ruins       0.038932
Fisico            -0.070915
Social             0.043308
Verbal             0.036620
Material          -0.084025
Virtual            0.009990
Nota-exatas       -0.116599
Nota-humanas      -0.005207
Nota-biológicas    0.056707
Bullying Total     0.010582
Name: Ano, dtype: float64

In [ ]:
skim(df)

In [39]:
#BOXPLOT

fig = px.box(df_copy, x="Sofreu Bullying?", y="Nota-exatas", color="Relação com Amigos", points='all')
fig.show()

In [42]:
fig = px.scatter(df_copy, x="Sofreu Bullying?", y="Nota-exatas", color="Relação com Amigos", symbol="Relação com Amigos")
fig.show()

In [61]:
ff.create_distplot([ df_copy['Nota-exatas'] ], group_labels=['Normal'])